In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import json
 
# Paths to the dataset directories
train_dir = 'train'
valid_dir = 'valid'
test_dir = 'test'
 
# Image dimensions
img_height, img_width = 640, 640
batch_size = 16
 
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
 
valid_datagen = ImageDataGenerator(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)
 
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)
 
# Number of classes
num_classes = len(train_generator.class_indices)
 
# Load MobileNetV2 model with pre-trained weights and without top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
 
# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
 
# Define the model
model = Model(inputs=base_model.input, outputs=predictions)
 
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False
 
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
 
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator
)
 
# Save the model
model.save('staff_mobilenet_v2_model.h5')
 
# Save class names
class_names = list(train_generator.class_indices.keys())
with open('class_names.json', 'w') as f:
    json.dump(class_names, f)
 
print("Model and class names saved successfully.")

In [1]:
# check_versions.py
import streamlit as st
import numpy as np
from PIL import Image
import tensorflow as tf
import scipy as sc
 
def check_versions():
    # Streamlit version
    st_version = st.__version__
    print(f"Streamlit version: {st_version}")
 
    # NumPy version
    np_version = np.__version__
    print(f"NumPy version: {np_version}")
 
    # Pillow (PIL) version
    pil_version = Image.__version__
    print(f"Pillow version: {pil_version}")
 
    # TensorFlow version
    tf_version = tf.__version__
    print(f"TensorFlow version: {tf_version}")

    # TensorFlow version
    sc_version = sc.__version__
    print(f"Scipy version: {sc_version}")
 
if __name__ == "__main__":
    check_versions()

Streamlit version: 1.37.0
NumPy version: 1.26.4
Pillow version: 10.4.0
TensorFlow version: 2.17.0
Scipy version: 1.13.1
